## Scientific computing and visualization
### Week 2 class & homework excercises
#### Juan Felipe Cerón Uribe - 201226725

### Day 5
#### Challenge 1
Adding a new Filter+Mapper+Actor to visualize the grid. The output of the completed challenge was the following image:
<img src="challenge1sess5.png">

In [3]:
import vtk
#help(vtk.vtkRectilinearGridReader())

rectGridReader = vtk.vtkRectilinearGridReader()
rectGridReader.SetFileName("data/jet4_0.500.vtk")
# do not forget to call "Update()" at the end of the reader
rectGridReader.Update()

rectGridOutline = vtk.vtkRectilinearGridOutlineFilter()
rectGridOutline.SetInputData(rectGridReader.GetOutput())

# New vtkRectilinearGridGeometryFilter() goes here:
plane = vtk.vtkRectilinearGridGeometryFilter()
plane.SetInputData(rectGridReader.GetOutput())

rectGridOutlineMapper = vtk.vtkPolyDataMapper()
rectGridOutlineMapper.SetInputConnection(rectGridOutline.GetOutputPort())

rectGridGeomMapper = vtk.vtkPolyDataMapper()
rectGridGeomMapper.SetInputConnection(plane.GetOutputPort())

outlineActor = vtk.vtkActor()
outlineActor.SetMapper(rectGridOutlineMapper)
outlineActor.GetProperty().SetColor(0.1, 0.1, 0.1)

gridGeomActor = vtk.vtkActor()
gridGeomActor.SetMapper(rectGridGeomMapper)
# Find out how to visualize this as a wireframe 
gridGeomActor.GetProperty().SetRepresentationToWireframe()
gridGeomActor.GetProperty().SetColor(0, 0, 0)

# Render of the filtered points
renderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(renderer)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
 
renderer.AddActor(outlineActor)
renderer.AddActor(gridGeomActor)
renderer.SetBackground(255/255,78/255,179/255)
renderer.ResetCamera()
renderer.GetActiveCamera().Elevation(60.0)
renderer.GetActiveCamera().Azimuth(30.0)
renderer.GetActiveCamera().Zoom(1.0)
 
renWin.SetSize(300, 300)
 
# interact with data
renWin.Render()
iren.Start()

#### Challenge 2
Use the vtkCalculator to compute vector magnitude.
<img src="challenge2sess5.png">

In [4]:
magnitudeCalcFilter = vtk.vtkArrayCalculator()
magnitudeCalcFilter.SetInputConnection(rectGridReader.GetOutputPort())
magnitudeCalcFilter.AddVectorArrayName('vectors')
# Set up here the array that is going to be used for the computation ('vectors')
magnitudeCalcFilter.SetResultArrayName('magnitude')
# Set up here the function that calculates the magnitude of a vector
magnitudeCalcFilter.SetFunction("mag(vectors)")
magnitudeCalcFilter.Update()

print(magnitudeCalcFilter.GetOutput())

vtkRectilinearGrid (0000019B2F8F7C40)
  Debug: Off
  Modified Time: 12615522
  Reference Count: 2
  Registered Events: (none)
  Information: 0000019B2FB46F40
  Data Released: False
  Global Release Data: Off
  UpdateTime: 12615523
  Field Data:
    Debug: Off
    Modified Time: 12615501
    Reference Count: 1
    Registered Events: (none)
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
  Number Of Points: 4194304
  Number Of Cells: 4112895
  Cell Data:
    Debug: Off
    Modified Time: 12615509
    Reference Count: 1
    Registered Events: 
      Registered Observers:
        vtkObserver (0000019B2FCE7870)
          Event: 33
          EventName: ModifiedEvent
          Command: 0000019B2FB47DA0
          Priority: 0
          Tag: 1
    Number Of Arrays: 0
    Number Of Components: 0
    Number Of Tuples: 0
    Copy Tuple Flags: ( 1 1 1 1 1 0 1 1 )
    Interpolate Flags: ( 1 1 1 1 1 0 0 1 )
    Pass Through Flags: ( 1 1 1 1 1 1 1 1 )
    Scalars: (none)
   

#### Challenge 3
Visualize the data set as colored points based on the "magnitude" value.

The parameter 'SetOnRatio(n)' sets the granularity of the render: one of every n points are taken into account.

The different `SetRandomType` options define the randomization algorithm used to select the data to be displayed.

The method `SetScalarModeToUsePointData()` tells the `pointsMapper` object to use point data in defining point colors instead of cell or field data.

Excecution result:
<img src="challenge3sess5.png">

In [6]:
#Extract the data from the result of the vtkCalculator
points = vtk.vtkPoints()
grid = magnitudeCalcFilter.GetOutput()
grid.GetPoints(points)
scalars = grid.GetPointData().GetArray('magnitude')

#Create an unstructured grid that will contain the points and scalars data
ugrid = vtk.vtkUnstructuredGrid()
ugrid.SetPoints(points)
ugrid.GetPointData().SetScalars(scalars)

#Populate the cells in the unstructured grid using the output of the vtkCalculator
for i in range (0, grid.GetNumberOfCells()):
    cell = grid.GetCell(i)
    ugrid.InsertNextCell(cell.GetCellType(), cell.GetPointIds())
    
#There are too many points, let's filter the points
subset = vtk.vtkMaskPoints()
subset.SetOnRatio(500)
#subset.RandomModeOn()
subset.SetInputData(ugrid)

#Make a vtkPolyData with a vertex on each point.
pointsGlyph = vtk.vtkVertexGlyphFilter()
pointsGlyph.SetInputConnection(subset.GetOutputPort())
#pointsGlyph.SetInputData(ugrid)
pointsGlyph.Update()

pointsMapper = vtk.vtkPolyDataMapper()
pointsMapper.SetInputConnection(pointsGlyph.GetOutputPort())
pointsMapper.SetScalarModeToUsePointData()

pointsActor = vtk.vtkActor()
pointsActor.SetMapper(pointsMapper)

# Render of the filtered points
renderer = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(renderer)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
 
renderer.AddActor(pointsActor)
renderer.AddActor(outlineActor)
renderer.SetBackground(255/255,78/255,179/255)
renderer.ResetCamera()
renderer.GetActiveCamera().Elevation(60.0)
renderer.GetActiveCamera().Azimuth(30.0)
renderer.GetActiveCamera().Zoom(1.0)
 
renWin.SetSize(300, 300)
 
# interact with data
renWin.Render()
iren.Start()

### Challenge 4
Visualize the data set as isosurfaces based on the "magnitude" value.

The `GenerateValues` function receives a range object and the number of contour lines that we wish to visualize and returns equally spaced contour values, that is, the values of the magnitude function along each of the isosurfaces. A larger integer parameter will result in more isosurfaces to be visualized. Changing the `scalarRange` parameter will result in changing the range from which the isosurface values will come from.

Excecution result:
<img src="challenge4sess5.png">

In [7]:
scalarRange = ugrid.GetPointData().GetScalars().GetRange()

#filter
isoFilter = vtk.vtkContourFilter()
isoFilter.SetInputData(ugrid)
isoFilter.GenerateValues(10, scalarRange)

#mapper
isoMapper = vtk.vtkPolyDataMapper()
isoMapper.SetInputConnection(isoFilter.GetOutputPort())

#actor
isoActor = vtk.vtkActor()
isoActor.SetMapper(isoMapper)
isoActor.GetProperty().SetOpacity(0.2)

#Option 1: Default vtk render window
renderer = vtk.vtkRenderer()
renderer.SetBackground(255/255,78/255,179/255)
renderer.AddActor(outlineActor)
renderer.AddActor(isoActor)
renderer.ResetCamera()

renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindow.SetSize(500, 500)
renderWindow.Render()

iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renderWindow)
iren.Start()

#### Homework
Design your own transfer function to highlight relevant features in the data set "wind.vti".

I found online that the wind speed limit for safe flight on a certain type of helicopter is 50km/h. Thus, assuming the speed data to be given in km/h I highlighted in blue the speed values near the limit, starting at around 40km/h. The highest speed value in the dataset is set to color completely red. Other colors are interpolated between those. Additionally I used an opacity transfer function to make irrelevant (low speed) values transparent.

<img src="home5.png">

In [1]:
import vtk

#------------READER ----------------------
reader=vtk.vtkXMLImageDataReader()
#set this file name!!
reader.SetFileName("data/wind_image.vti")
reader.Update()
print("Range of the \'wind_speed\' scalars: "+str(reader.GetOutput().GetPointData().GetScalars().GetRange(-1)))
#xmi,xma,ymi,yma,zmi,zma=reader.GetOutput().GetBounds()
#------------END READER ------------------

##------------FILTER, MAPPER, AND ACTOR: VOLUME RENDERING -------------------
# Create transfer mapping scalar value to opacity
opacityTransferFunction = vtk.vtkPiecewiseFunction()
opacityTransferFunction.AddPoint(30, 0)
opacityTransferFunction.AddPoint(50, 0.1)
opacityTransferFunction.AddPoint(79, 0.5)

# Create transfer mapping scalar value to color
colorTransferFunction = vtk.vtkColorTransferFunction()
colorTransferFunction.AddRGBPoint(44,0,0,1)
colorTransferFunction.AddRGBPoint(60,1,0,0)

# The property describes how the data will look
volumeProperty = vtk.vtkVolumeProperty()
volumeProperty.SetColor(colorTransferFunction)
volumeProperty.SetScalarOpacity(opacityTransferFunction)
volumeProperty.ShadeOff()
volumeProperty.SetInterpolationTypeToLinear()

# The mapper / ray cast function know how to render the data
#volumeMapper = vtk.vtkProjectedTetrahedraMapper()
#volumeMapper = vtk.vtkUnstructuredGridVolumeZSweepMapper()
volumeMapper = vtk.vtkGPUVolumeRayCastMapper()
#volumeMapper = vtk.vtkUnstructuredGridVolumeRayCastMapper()
volumeMapper.SetInputData(reader.GetOutput())

# The volume holds the mapper and the property and
# can be used to position/orient the volume
volume = vtk.vtkVolume()
volume.SetMapper(volumeMapper)
volume.SetProperty(volumeProperty)

##------------END VOLUME RENDERING ----------------------

#outline
outline = vtk.vtkOutlineFilter()
outline.SetInputData(reader.GetOutput())
outlineMapper = vtk.vtkPolyDataMapper()
outlineMapper.SetInputConnection(outline.GetOutputPort())
outlineActor = vtk.vtkActor()
outlineActor.SetMapper(outlineMapper)
outlineActor.GetProperty().SetColor(1,1,1)

#---------RENDERER, RENDER WINDOW, AND INTERACTOR ----------
renderer = vtk.vtkRenderer()
renderer.SetBackground(0.1, 0.1, 0.1)
renderer.AddVolume(volume)
renderer.AddActor(outlineActor)
renderer.ResetCamera()

renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindow.SetSize(500, 500)
renderWindow.Render()

iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renderWindow)
iren.Start()
#---------END RENDERER, RENDER WINDOW, AND INTERACTOR -------

Range of the 'wind_speed' scalars: (0.2681176960468292, 78.90724182128906)


### Day 6
Exploration of the `wind` dataset using paraview.

First I applied streamtracer and ribbon filters in order to visualize particle trayectories. Notice how this technique reveals a vortex near the bottom-right corner.

<img src="para1.png">

Next I applied a glyph filter on a particular slice of interest to visualize wind velocity vectors' direction, color coded by velocity magnitude:

<img src="para2.png">

Finally I applied threshold filters to detect the top and bottom 10% wind speeds in the dataset.

<img src="para3.png">

Complete the scripts provided during the lab session. I added a transfer function and lookup table to the `07_NB_Challenge_1.py` script:

<img src="chall7.png">

In [ ]:
import vtk

reader = vtk.vtkRectilinearGridReader()
reader.SetFileName("c:/users/bananin/desktop/SS_2017/data/jet4_0.500.vtk")
reader.Update()
output = reader.GetOutput()

xmi, xma, ymi, yma, zmi, zma = output.GetBounds()

# Color Transfer Function and LookUpTable
# Create transfer mapping scalar value to color
colorTransferFunction = vtk.vtkColorTransferFunction()
colorTransferFunction.AddRGBPoint(0.0, 0, 0.0, 1)
colorTransferFunction.AddRGBPoint(0.2, 1, 1, 0)

tableSize = 100

lut = vtk.vtkLookupTable()
lut.SetNumberOfTableValues(tableSize)
lut.Build()

for i in range(0,tableSize):
    rgb = list(colorTransferFunction.GetColor(float(i)/tableSize))+[0.5]
    lut.SetTableValue(i,rgb)
        

# A plane for the seeds
plane = vtk.vtkPlaneSource()
plane.SetOrigin(0, 0, 0)
plane.SetPoint1(xma, 0, 0)
plane.SetPoint2(0, 0, zma)
plane.SetXResolution(20)
plane.SetYResolution(20)

# Add the outline of the plane
outline = vtk.vtkOutlineFilter()
outline.SetInputData(plane.GetOutput())
outlineMapper = vtk.vtkPolyDataMapper()
outlineMapper.SetInputConnection(outline.GetOutputPort())
outlineActor = vtk.vtkActor()
outlineActor.SetMapper(outlineMapper)
outlineActor.GetProperty().SetColor(1,1,1)

# Compute streamlines
streamline = vtk.vtkStreamTracer()
streamline.SetSourceConnection(plane.GetOutputPort())
streamline.SetInputConnection(reader.GetOutputPort())
# Try different integration alternatives! See the documentation of vtkStreamTracer
streamline.SetIntegrationDirectionToForward()
streamline.SetMaximumPropagation(1)
streamline.SetComputeVorticity(True)

# Pass the streamlines to the mapper
streamlineMapper = vtk.vtkPolyDataMapper()
streamlineMapper.SetLookupTable(lut)
streamlineMapper.SetInputConnection(streamline.GetOutputPort())
streamlineMapper.SetScalarVisibility(True)
streamlineMapper.SetScalarModeToUsePointFieldData()
streamlineMapper.SelectColorArray('vectors')
streamlineMapper.SetLookupTable(lut)
# See documentation for the parameter in GetRange()
# http://www.vtk.org/doc/nightly/html/classvtkDataArray.html#ab7efccf59d099c038a4ae269a490e1a3
streamlineMapper.SetScalarRange((reader.GetOutput().GetPointData().GetVectors().GetRange(-1)))

# Pass the mapper to the actor
streamlineActor = vtk.vtkActor()
streamlineActor.SetMapper(streamlineMapper)
streamlineActor.GetProperty().SetLineWidth(2.0)

# Add the outline of the data set
gOutline = vtk.vtkRectilinearGridOutlineFilter()
gOutline.SetInputData(output)
gOutlineMapper = vtk.vtkPolyDataMapper()
gOutlineMapper.SetInputConnection(gOutline.GetOutputPort())
gOutlineActor = vtk.vtkActor()
gOutlineActor.SetMapper(gOutlineMapper)
gOutlineActor.GetProperty().SetColor(1,1,1)

# Rendering / Window
renderer = vtk.vtkRenderer()
renderer.SetBackground(0.0, 0.0, 0.0)
renderer.AddActor(streamlineActor)
renderer.AddActor(outlineActor)
renderer.AddActor(gOutlineActor)

renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindow.SetSize(500, 500)
renderWindow.Render()

interactor = vtk.vtkRenderWindowInteractor()
interactor.SetInteractorStyle(vtk.vtkInteractorStyleTrackballCamera())
interactor.SetRenderWindow(renderWindow)
interactor.Initialize()
interactor.Start()


### Day 7
#### Homework
Edit the `step_wave` function in the `08_PDE_part1.py` script to simulate a stationary wave. The solution is in the `pde_challenge.py` script. It must be excecuted from the command line for the animation to show. The excecution shows a standing wave like the following:

<img src="pde_challenge.PNG">